In [1]:
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Together
from langchain_core.prompts import PromptTemplate

In [2]:
# loader = PyMuPDFLoader("../DocumentStore/msme-features.pdf")
# loader = PyMuPDFLoader("../DocumentStore/UMK-FInal updated User manual Create org-260424-093614.pdf")
loader = PyMuPDFLoader("../DocumentStore/FRDs-170424-070159.pdf")
pages = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap = 200, 
                                                length_function=len,
                                                is_separator_regex=False,
                                                separators=[
                                                            "\n\n",
                                                            "\n",
                                                            ".",
                                                            ",",
                                                            "\u200B",  # Zero-width space
                                                            "\uff0c",  # Fullwidth comma
                                                            "\u3001",  # Ideographic comma
                                                            "\uff0e",  # Fullwidth full stop
                                                            "\u3002",  # Ideographic full stop
                                                            " ",
                                                            "",
                                                        ]
                                            )
splits = text_splitter.split_documents(pages)

In [15]:
# splits[1].metadata

In [5]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")

c:\Users\debgh\AllProjects\RAGProjects\pdfBot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function, persist_directory="../chroma_d")

In [7]:
query = "Bank Reconcilation Statement"
k = 10
results = vectorstore.similarity_search_with_relevance_scores(query, k=k)
# Print out just the video titles
# [(r[0].metadata['title'], r[1]) for r in results]

In [8]:
query = "Bank Reconcilation Statement"
k = 10
results = vectorstore.similarity_search_with_relevance_scores(query, k=k)

In [9]:
# print(results[0])

In [13]:
k = 5
retriever = vectorstore.as_retriever(search_kwargs={"k":k})
query = "How update MSME UDYAM details?"
results = retriever.invoke(query)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# print(results[0].page_content)

In [11]:
# prompt = hub.pull("rlm/rag-prompt")

prompt = PromptTemplate(input_variables=['context', 'question'],
                        template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep answers descriptive and mention the process.\nQuestion: {question} \nContext: {context} \nAnswer:")

# model = "meta-llama/Llama-2-7b-chat-hf"
# response = Together(model="mistralai/Mixtral-8x22B-Instruct-v0.1", 
#                     temperature = 0.3,
#                     together_api_key= "dc50d15b1bed2012efe79427bafd87b100c9bf0d20609c832c485aec433ddd7d", 
#                     max_tokens = 2000)

response = Together(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        together_api_key= "dc50d15b1bed2012efe79427bafd87b100c9bf0d20609c832c485aec433ddd7d",
        temperature=0.3,
        max_tokens=512
    )

rag_chain = (
    {"context": retriever 
    | format_docs, "question": RunnablePassthrough()}
    | prompt
    | response
    | StrOutputParser())
# print(prompt)

c:\Users\debgh\AllProjects\RAGProjects\pdfBot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 0.2. An updated version of the class exists in the langchain-together package and should be used instead. To use it run `pip install -U langchain-together` and import as `from langchain_together import Together`.
  warn_deprecated(


In [12]:
query = "What is the functionality of Reconcilation statement:missing in books?"

query_list = ["Describe the ledger statement data-table?", ]
# query = "Summarise the document for me in 500 words"
ls = rag_chain.invoke(query)
print(ls)


The functionality of Reconciliation statement: Missing in Books is to identify and report on missing transactions in the Bank Reconciliation Statement (BRS) process. It is a component of the BRS process that displays transactions with a "Missing in Books" status from the Bank Statement within a specified date range. The system dynamically searches and displays transactions matching the user's input and filters records based on voucher type. The section functions similarly to the Bank Statement screen within the BRS, excluding the "Status" column and offering a voucher type filter. The Reconciliation statement: Missing in Books section is a part of the BRS process that helps identify and report on missing transactions.
